In [1]:
import os
os.chdir('..')

### Keras
Can we define the custom metric in Keras?

In [5]:
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.metrics import f1_score

def f1_macro(y_true, y_pred):
    
    y_true = K.cast(y_true, tf.float64)
    y_pred = K.cast(y_pred, tf.float64)
    
    TP = K.sum(y_true * K.round(y_pred), axis=0)
    FN = K.sum(y_true * (1 - K.round(y_pred)), axis=0)
    FP = K.sum((1 - y_true) * K.round(y_pred), axis=0)
    
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    
    # Convert NaNs to Zero
    prec = tf.where(tf.is_nan(prec), tf.zeros_like(prec), prec)
    rec = tf.where(tf.is_nan(rec), tf.zeros_like(rec), rec)
    
    f1 = 2 * (prec * rec) / (prec + rec)
    
    # Convert NaN to Zero
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    f1 = K.mean(f1)
    
    return f1

In [9]:
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, Flatten
from tensorflow.keras.layers import MaxPooling2D

def build_model():
    """Builds the testModel1 Architecture.
    """
    inputs = Input(shape=(264, 264, 4))
    x = Conv2D(64, (3, 3), activation=tf.nn.relu)(inputs)
    x = MaxPooling2D((2, 2))(x)

    for i in range(2):
        x = Conv2D(64, (3, 3), activation=tf.nn.relu)(x)
        x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(264, activation=tf.nn.relu)(x)
    outputs = Dense(28, activation=tf.nn.sigmoid)(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    optimizer = tf.train.AdamOptimizer(0.001)

    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics={'f1_macro' : f1_macro})
    
    return model

In [10]:
model = build_model()

### Set up Data
For now we're just using some random numbers rather than real data, since we just want to know if the custom metric is actually being calculated.

In [11]:
import numpy as np
random_data_train = np.random.rand(100, 264, 264, 4)
val_data_train = np.random.rand(100, 264, 264, 4)


true_labels_train = np.round(np.random.rand(100, 28))
val_label_train = np.round(np.random.rand(100, 28))

In [12]:
history = model.fit(random_data_train, true_labels_train, validation_data=(val_data_train, val_label_train))

Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 1.2438 - val_loss: 0.7157


#### Where the hell is the F1-Macro calculation?

In [13]:
history.history

{'val_loss': [0.7156828784942627], 'loss': [1.2438363027572632]}

In [14]:
model.metrics

{'f1_macro': <function __main__.f1_macro(y_true, y_pred)>}

... The metric is there... but where is it?

In [15]:
model.evaluate(random_data_train, true_labels_train)

100/100 [==============================] - 1s 5ms/step


0.7052780342102051

That's the loss, but the metric isn't there.

### Verify Custom Metric Calculates Stuff 

In [17]:
preds = model.predict(random_data_train)
rounded_preds = np.round(preds)

In [18]:
## From sklearn
f1_score(true_labels_train, rounded_preds, average='macro')

/home/matt/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3206456974113276

In [19]:
## Custom
f1 = f1_macro(true_labels_train, rounded_preds)

In [20]:
K.eval(f1)

0.32064569741132765

Yup, identical.